<a href="https://colab.research.google.com/github/rafaelwinter01/google-ai-embedding/blob/main/embedings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [2]:
import numpy as np
import pandas as pd

import google.generativeai as genai

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
# Exemplo de embedding
title = "O que são embeddings?"
sample_text = ("Embeddings de texto são uma técnica de processamento de linguagem natural (PLN)"
"que converte texto em vetores numéricos."
"\n"
"Os embeddings capturam significado semântico e contexto, resultando em um texto com significados semelhantes com embeddings mais próximos.")

embeddings = genai.embed_content(model="models/embedding-001", content=sample_text, title=title, task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [-0.039487652, -0.01880299, 0.0018553657, 0.06945675, 0.018302418, 0.029230395, 0.010455392, 0.025647705, 0.0060858345, 0.031031141, -0.019510994, -0.010414433, -0.011344551, -0.026164616, -0.02056451, -0.03663317, -5.068075e-05, 0.030182395, 0.02690947, -0.038689848, -0.027742978, 0.011586352, 0.01798695, -0.031611167, 0.004754842, -0.02220197, 0.016220486, -0.031031046, -0.018483013, 0.033298396, -0.030670077, 0.05102797, -0.014069758, 0.0062100906, 0.007071411, 0.0010829094, 0.008607535, -0.057207078, 0.00852856, 0.006606933, 0.017518157, -0.03657026, -0.007914115, -0.02405125, -0.010484683, -0.009695199, 0.01728655, 0.0040700054, -0.017300326, -0.09501878, -0.014753452, -0.040441226, 0.09177182, 0.0004956722, -0.046404265, -0.039460678, 0.036882326, -0.026971644, -0.05305227, -0.0143573675, -0.00025492965, -0.031143872, 0.008354004, 0.010340668, -0.024938378, 0.005401439, -0.02484716, 0.018198647, 0.037857953, -0.027754586, 0.017478274, -0.026970496, 0.028422354, 0.00

In [6]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [19]:
df = pd.DataFrame(documents)
# df.colums = ["t", "c"]
df

,title,content
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [8]:
model = "models/embedding-001"

In [20]:
def embed_fn(title, text):
  return genai.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [22]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["title"], row["content"]), axis=1)
df

,title,content,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [25]:
def gen_find_search(search, base, model):
  emb = genai.embed_content(model=model, content=search, task_type="RETRIEVAL_QUERY")
  esc_prod = np.dot(np.stack(df['Embeddings']), emb["embedding"])
  index = np.argmax(esc_prod)
  return df.iloc[index]["content"]

In [26]:
example = "how to shift gears in a google car?"
part = gen_find_search(example, df, model)
print(part)

Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.


In [31]:
generation_config = {
    "temperature": 0,
    "candidate_count": 1
}

In [32]:
prompt = f"Rewrite this text in a fun way, without adding any information from outside the text: {example}"
model_2 = genai.GenerativeModel("gemini-1.0-pro",generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Step 1: Get Your Groove On**

Hop into your Google car and buckle up, it's time to shift gears like a pro!

**Step 2: The Magic Wand**

Spot that sleek little lever on the right? That's your gear selector, the key to unlocking the power of your ride.

**Step 3: The "D" Zone**

For cruising around town, slide the lever into the "D" zone. It's like putting your car on autopilot, letting it handle the shifting for you.

**Step 4: The "R" Revolution**

Need to reverse? Give the lever a gentle nudge into the "R" zone. It's like a time machine, taking you back in style.

**Step 5: The "P" Party**

When you're ready to park, shift into "P" for "Party Time!" It's like giving your car a well-deserved break.

**Step 6: The "N" Neutral**

If you're feeling adventurous, try "N" for "Neutral." It's like letting your car take a deep breath before the next adventure.

**Bonus Tip: The "B" Boost**

For those extra-steep hills, engage "B" for "Boost." It's like giving your car a turbocharged boost to